# CRIM Intervals:  Cadences

### Reminders:

#### Import Music Files

* If you are exploring pieces from CRIM, importing simply involves providing the CRIM URL of the MEI file:  
    * **`piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')`**

* But you can also use the Notebook with any MEI, MusicXML, or MIDI file of your own. You can easily do this when you run the Notebooks on Jupyter Hub, you will also find a folder called **`Music_Files`**.  Upload the file here, then provide the path to that file: 
    * **`piece = importScore('Music_Files/My_File_Name.mei')`**.  

#### Save outputs as CSV or Excel

* The Jupyter Hub version of these Notebooks also provides a folder called **`saved_csv`**.  You can save **csv** files of any data frame there with this command: 
    * **`notebook_data_frame_name.to_csv('saved_csv/your_file_title.csv')`**.
* If you prefer **Excel** documents (which are better for anything with a complex set of columns or hierarhical index), use **ExcelWriter**.  In the following code, you will need to provide these commands:
    * **`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')`**
* Now convert your dataframe to Excel
    * **`frame_name.to_excel(writer, sheet_name='Sheet1')`**
* And finally save the new file to the folder here in the Notebook:
    * **`writer.save()`**

Put the following code to a new cell and update the frame_name and file_name:

`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')` <br>
`frame_name.to_excel(writer, sheet_name='Sheet1')` <br>
`writer.save()` <br>


## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [3]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)
# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [2]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
piece.score.metadata.title

Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei


'Ave Maria'

## C. Find Cadences with Modular Analysis

Alex Morgan has built a powerful tool that identifies cadences according to the combinations of two-voice **modules** that describe the typical contrapuntal motion between the various **cadential voice functions** (**CVF**) heard in Renaissance polyphony:  **cantizans and tenorizans**, **cantizans and bassiszans**, etc.

The tool uses modular analysis to identify **conjunctions** of these pairs in order to predict cadences of various kinds. But there are many combinations, especially once we consider that voices functions (or roles) can be **displaced** (as when the tenorizans role appears in the Superius part and the cantizans appears in the Tenor part), or through **irregular** motion, and even **interrupted**, as when a voice is suddenly silent. 

You can in fact check all of the cadential voice functions (CVFs) for a given piece below.  But Alex's system also conveniently **labels** the cadences according to **type**, **tone**, **evaded** and also provides information about the relative place within the piece, the adjacent cadences, and many other features, too.

Note:  **Measure** and **Beat** columns are in the body of the table, not at the Index.

**Column Headings Explained**:

* The **Low** and **Tone** columns give the pitches of the **lowest sounding pitch (in any voice) at the perfection**, and the **goal tone of the cantizans** (or altizans if there is no cantizans) respectively.

* **RelLow** is the lowest pitch of each cadence shown as an interval measured against the last pitch in the **Low** column. Likewise, **RelTone** is the cadential tone shown as an interval measured against the last pitch in the **Tone** column.

* The **SinceLast** and **ToNext** columns are the time in quarter notes since the last or to the next cadence.

* The **Progress** column is a relative indication of position in the piece.  **0** is the beginning of the piece; **1.0** is the end of the piece.



Read more via the documentation: **`print(piece.classifyCadences.__doc__)`**

View the **Cadential Voice Function** and **Cadence Label** tables here:  https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/.  These can easily be updated with revised or new cadence types.

### C.1 Classify the Cadences

In [3]:
cadences = piece.classifyCadences()
col_list = ['Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
cadences = cadences[col_list]
cadences

,CadType,Evaded,LeadingTones,Low,RelLow,Tone,RelTone,Measure,Beat,Progress,SinceLast,ToNext
188.0,NaN,NaN,NaN,E3,-m3,G,P1,24,3.0,0.145963,188.0,38.0
226.0,Clausula Vera,True,1.0,A2,-m7,C,-P5,29,2.0,0.175466,38.0,6.0
232.0,Authentic,True,1.0,A2,-m7,C,-P5,30,1.0,0.180124,6.0,40.0
272.0,Phrygian,False,1.0,E4,M6,E,-m3,35,1.0,0.211180,40.0,32.0
304.0,Phrygian,False,1.0,E3,-m3,E,-m3,39,1.0,0.236025,32.0,112.0


### C.2  Check Voice Functions for all Cadences of a Given Piece

When **return_type** is set to **'functions'** (or just 'f' for short), a table
of the **cadential voice functions** (CVF) is returned. Each CVF is
represented with a single-character label with the meanings as follows:

* `C`: **cantizans motion up a step** (can also be ornamented e.g. Landini)
* `T`: **tenorizans motion down a step** (can be ornamented with anticipations)
* `B`: **bassizans motion up a fourth or down a fifth**
* `A`: **altizans motion, similar to cantizans**, but cadences to a fifth
above a tenorizans instead of an octave  <br><br>
* `L`: **leaping contratenor** motion up an octave at the perfection
* `P`: **plagal bassizans** motion up a fifth or down a fourth <br><br>
* `c`: **evaded cantizans** when it moves to an unexpected note at the perfection
* `t`: **evaded tenorizans** when it goes up by step at the perfection
* `b`: **evaded bassizans** when it goes up by step at the perfection
* `u`: **evaded bassizans** when it goes down by third at the perfection
(there are no evaded labels for the altizans, plagal bassizans leaping
contratenor CVFs) <br><br>
* `x`: evaded **bassizans motion where the voice drops out** at the perfection
* `y`: evaded **cantizans motion where the voice drops out** at the perfection
* `z`: evaded **tenorizans motion where the voice drops out** at the perfection


View the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

In [4]:
cvf = piece.classifyCadences(return_type='f')
piece.detailIndex(cvf, offset=True).fillna('-')


,,,[Superius],Altus,Tenor,Bassus
Measure,Beat,Offset,,,,
24,3.0,188.0,-,T,C,b
29,2.0,226.0,-,t,C,-
30,1.0,232.0,C,-,T,b
35,1.0,272.0,C,T,-,-
39,1.0,304.0,-,A,C,T
53,1.0,416.0,C,-,T,B
60,1.0,472.0,C,T,B,-
65,1.0,512.0,-,-,C,T
77,1.0,608.0,C,-,T,B


### C.3.  Check for Missed Cadences (One or More Pieces)


* The resulting table lists the **Cadential Voice Functions** (CVF's) for each of the probable cadences that the Cadence Classifier was **unable to categorize** according to the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

* See below for additional instructions.

In [5]:

corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei']) # add as many pieces as you want here, each in single quotations, separated by commas
cad_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=False)
cvf_dfs = corpus.batch(ImportedPiece.classifyCadences, kwargs={'return_type': 'f'})
missed = []
for i, cad in enumerate(cad_dfs):
    df = cvf_dfs[i].loc[cad.CadType.isnull(), :]
    df = df[df.columns[range(-2, len(df.columns) -2)]]
    df.columns = range(len(df.columns))
    missed.append(df)
result = pd.concat(missed)
rm = piece.detailIndex(result, offset=True)
result_offset_list = rm.index.get_level_values('Offset')
rm

Memoized piece detected.


,,,0,1,2,3,4,5
Measure,Beat,Offset,,,,,,
24,3.0,188.0,Josquin Des Prés,Ave Maria,NaN,T,C,b


### C.4.  Check the Harmonic Modules of Missing Cadences

* For a given piece (since this tool works for one composition at a time), you can check the **harmonic modules** (for example `7_Held, 6_-2, 8` for a cantizans-tenorizans pair) at any given point in your piece
    * **Import the piece** (just ONE at a time)
    * Run the **"missed cadence finder"** (Section D) above.  You must run this with just ONE piece!
* The default **modular ngram** is 3 events long.  Adjust **`n`** as needed for longer modules
* The default **interval type** is diatonic ('d').  Adjust **`kind`** as needed for chromatic ('c')

Use the results to report the new cadential voice functions and label, as explained above.

In [6]:
piece_har = piece.getHarmonic(kind='d', compound=False)
ngrams = piece.getNgrams(df=piece_har, n=3, how='modules', exclude=['Rest'], offsets='last')
ngrams_filtered = ngrams.loc[result_offset_list,:].fillna("-")
ngrams_filtered

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
Offset,,,,,,
188.0,"4_Held, 3_2, 3","8_2, 5_2, 3",-,"-7_-2, -6_2, -8",-,-


### C.5.  Inventory of Cadences Found:  One or Many Pieces at Once

* Here you can report an inventory of cadences by **type** and **tone** (and **evaded** status)


* To search multiple pieces at once (each returning its own dataframe of results):  enter the urls, separated by commas (each url within **single quotation marks**, and separated from the previous by a **comma** ).  Thus:

* **`corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei', 'https://crimproject.org/mei/CRIM_Model_0010.mei'])`**

In [21]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0025.mei', 'https://crimproject.org/mei/CRIM_Model_0008.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=False)
summary = combined_df.groupby(['CadType', 'Tone', 'Evaded']).size().reset_index(name='counts')
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
combined_dfs = combined_df[col_list]
combined_dfs

Memoized piece detected.
Memoized piece detected.


,Composer,Title,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext
28.0,Cipriano de Rore,Ite rime dolenti,4,3.0,Clausula Vera,None,True,-1.0,A2,-P5,NaN,0.038462,28.0,26.0
54.0,Cipriano de Rore,Ite rime dolenti,7,4.0,Authentic,A,True,0.0,A3,P4,P4,0.074176,26.0,58.0
112.0,Cipriano de Rore,Ite rime dolenti,15,1.0,NaN,None,NaN,NaN,A2,-P5,NaN,0.153846,58.0,42.0
154.0,Cipriano de Rore,Ite rime dolenti,20,2.0,Clausula Vera,None,True,-1.0,D3,-M2,NaN,0.211538,42.0,54.0
208.0,Cipriano de Rore,Ite rime dolenti,27,1.0,Authentic,A,False,0.0,A2,-P5,P4,0.285714,54.0,316.0
524.0,Cipriano de Rore,Ite rime dolenti,65,3.0,Plagal,B,False,1.0,E3,P1,P5,0.719780,316.0,4.0
528.0,Cipriano de Rore,Ite rime dolenti,66,1.0,Authentic,None,True,-1.0,F4,m9,NaN,0.725275,4.0,124.0
652.0,Cipriano de Rore,Ite rime dolenti,81,3.0,Phrygian,E,False,1.0,E3,P1,P1,0.895604,124.0,92.0
188.0,Josquin Des Prés,Ave Maria,24,3.0,Authentic,G,True,1.0,E3,-m3,P1,0.145963,188.0,38.0
226.0,Josquin Des Prés,Ave Maria,29,2.0,Clausula Vera,C,True,1.0,A2,-m7,-P5,0.175466,38.0,6.0


A simpler summary, without tone or evaded status:

In [6]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['CadType'].value_counts()
print(cadTypeCounts)
# Get the number of cadences per Beat level:
cadBeatCounts = combined_df['Beat'].value_counts()
print(cadBeatCounts)


Memoized piece detected.
Memoized piece detected.
Clausula Vera    31
Phrygian         16
Authentic        12
Plagal            2
Name: CadType, dtype: int64
1.0    47
3.0    15
Name: Beat, dtype: int64


In [26]:
combined_dfs

,Composer,Title,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext,type
28.0,Cipriano de Rore,Ite rime dolenti,4,3.0,Clausula Vera,None,True,-1.0,A2,-P5,NaN,0.038462,28.0,26.0,NaN
54.0,Cipriano de Rore,Ite rime dolenti,7,4.0,Authentic,A,True,0.0,A3,P4,P4,0.074176,26.0,58.0,AuthenticA
112.0,Cipriano de Rore,Ite rime dolenti,15,1.0,NaN,None,NaN,NaN,A2,-P5,NaN,0.153846,58.0,42.0,NaN
154.0,Cipriano de Rore,Ite rime dolenti,20,2.0,Clausula Vera,None,True,-1.0,D3,-M2,NaN,0.211538,42.0,54.0,NaN
208.0,Cipriano de Rore,Ite rime dolenti,27,1.0,Authentic,A,False,0.0,A2,-P5,P4,0.285714,54.0,316.0,AuthenticA
524.0,Cipriano de Rore,Ite rime dolenti,65,3.0,Plagal,B,False,1.0,E3,P1,P5,0.719780,316.0,4.0,PlagalB
528.0,Cipriano de Rore,Ite rime dolenti,66,1.0,Authentic,None,True,-1.0,F4,m9,NaN,0.725275,4.0,124.0,NaN
652.0,Cipriano de Rore,Ite rime dolenti,81,3.0,Phrygian,E,False,1.0,E3,P1,P1,0.895604,124.0,92.0,PhrygianE
188.0,Josquin Des Prés,Ave Maria,24,3.0,Authentic,G,True,1.0,E3,-m3,P1,0.145963,188.0,38.0,AuthenticG
226.0,Josquin Des Prés,Ave Maria,29,2.0,Clausula Vera,C,True,1.0,A2,-m7,-P5,0.175466,38.0,6.0,Clausula VeraC


In [32]:
cadTypeCounts.to_frame()

,CadType
Clausula Vera,40
Authentic,11
Phrygian,7
Plagal,2


In [27]:
combined_dfs["type"] = combined_dfs["CadType"] + combined_dfs["Tone"]

/var/folders/_s/4t2p1z2x0yxcv068dtqj31tw0000gq/T/ipykernel_32056/2597064929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_dfs["type"] = combined_dfs["CadType"] + combined_dfs["Tone"]


In [28]:
combined_dfs

,Composer,Title,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext,type
28.0,Cipriano de Rore,Ite rime dolenti,4,3.0,Clausula Vera,None,True,-1.0,A2,-P5,NaN,0.038462,28.0,26.0,NaN
54.0,Cipriano de Rore,Ite rime dolenti,7,4.0,Authentic,A,True,0.0,A3,P4,P4,0.074176,26.0,58.0,AuthenticA
112.0,Cipriano de Rore,Ite rime dolenti,15,1.0,NaN,None,NaN,NaN,A2,-P5,NaN,0.153846,58.0,42.0,NaN
154.0,Cipriano de Rore,Ite rime dolenti,20,2.0,Clausula Vera,None,True,-1.0,D3,-M2,NaN,0.211538,42.0,54.0,NaN
208.0,Cipriano de Rore,Ite rime dolenti,27,1.0,Authentic,A,False,0.0,A2,-P5,P4,0.285714,54.0,316.0,AuthenticA
524.0,Cipriano de Rore,Ite rime dolenti,65,3.0,Plagal,B,False,1.0,E3,P1,P5,0.719780,316.0,4.0,PlagalB
528.0,Cipriano de Rore,Ite rime dolenti,66,1.0,Authentic,None,True,-1.0,F4,m9,NaN,0.725275,4.0,124.0,NaN
652.0,Cipriano de Rore,Ite rime dolenti,81,3.0,Phrygian,E,False,1.0,E3,P1,P1,0.895604,124.0,92.0,PhrygianE
188.0,Josquin Des Prés,Ave Maria,24,3.0,Authentic,G,True,1.0,E3,-m3,P1,0.145963,188.0,38.0,AuthenticG
226.0,Josquin Des Prés,Ave Maria,29,2.0,Clausula Vera,C,True,1.0,A2,-m7,-P5,0.175466,38.0,6.0,Clausula VeraC


In [29]:
pairs = combined_dfs.type.apply(lambda x: list(combinations(x, 2)))
pairs

TypeError: 'float' object is not iterable

In [ ]:
pairs2 = pairs.explode().dropna()
pairs2

In [ ]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

In [ ]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [ ]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [ ]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [ ]:
pyvis_graph.from_nx(G)

In [ ]:
pyvis_graph.show('MINT.html')